In [1]:
import torch
from torch import nn
from mmcls.models import ResNet

c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\mmcv\cnn\bricks\transformer.py:28: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


In [5]:
resnet = ResNet(50,strides=(1, 2, 1, 1),dilations=(1, 1, 2, 4))

In [6]:
a =torch.randn(1,3,224,224)
out =resnet(a)
print(out[0].shape)

torch.Size([1, 2048, 28, 28])


In [7]:
class PPM(nn.Module):
    def __init__(self,
                 pool_scales, 
                 in_channels,
                 channels, 
                 conv_cfg, 
                 norm_cfg,
                 act_cfg, 
                 align_corners):
        super(PPM, self).__init__()
        self.pool_scales = pool_scales
        self.align_corners = align_corners
        self.in_channels = in_channels
        self.channels = channels
        self.conv_cfg = conv_cfg
        self.norm_cfg = norm_cfg
        self.act_cfg = act_cfg
        for pool_scale in pool_scales:
            self.append(
                nn.Sequential(
                    nn.AdaptiveAvgPool2d(pool_scale),
                    ConvModule(
                        self.in_channels,
                        self.channels,
                        1,
                        conv_cfg=self.conv_cfg,
                        norm_cfg=self.norm_cfg,
                        act_cfg=self.act_cfg)))

    def forward(self, x):
        """Forward function."""
        ppm_outs = []
        for ppm in self:
            ppm_out = ppm(x)
            upsampled_ppm_out = resize(
                ppm_out,
                size=x.size()[2:],
                mode='bilinear',
                align_corners=self.align_corners)
            ppm_outs.append(upsampled_ppm_out)
        return ppm_outs